Build a regression model.

In [1]:
import numpy as np
import pandas as pd

In [ ]:
import statsmodels.api as sm

In [23]:
df = pd.read_csv("bentonville.csv")
df_fsqmodel = pd.read_csv("df_merged4sq.csv")
df_yelpmodel = pd.read_csv("df_mergedyelp.csv")
df_fsq_closest_station = pd.read_csv("df_fsq_closest_station.csv")
df_yelp_closest_station = pd.read_csv("df_yelp_closest_station.csv")
df_merged4sq = pd.read_csv("df_merged4sq.csv")
df_mergedyelp = pd.read_csv("df_mergedyelp.csv")

In [24]:
df_merged4sq.columns

Index(['fsq_id', 'categories', 'chains', 'closed_bucket', 'distance', 'link',
       'name_x', 'timezone', 'geocodes.main.latitude',
       'geocodes.main.longitude', 'geocodes.roof.latitude',
       'geocodes.roof.longitude', 'location.address', 'location.census_block',
       'location.country', 'location.dma', 'location.formatted_address',
       'location.locality', 'location.postcode', 'location.region',
       'location.address_extended', 'location.cross_street',
       'geocodes.drop_off.latitude', 'geocodes.drop_off.longitude',
       'geocodes.front_door.latitude', 'geocodes.front_door.longitude',
       'related_places.parent.fsq_id', 'related_places.parent.categories',
       'related_places.parent.name', 'station', 'related_places.children',
       'Unnamed: 0', 'name_y', 'longitude', 'latitude', 'free_bikes',
       'empty_slots'],
      dtype='object')

In [65]:
print(df_merged4sq.info())  # Check column types and missing values
print(df_merged4sq.describe())  # Get summary statistics


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129 entries, 0 to 128
Data columns (total 40 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   fsq_id                            129 non-null    object 
 1   categories                        129 non-null    object 
 2   chains                            129 non-null    object 
 3   closed_bucket                     129 non-null    object 
 4   distance                          129 non-null    int64  
 5   link                              129 non-null    object 
 6   name_x                            129 non-null    object 
 7   timezone                          112 non-null    object 
 8   geocodes.main.latitude            129 non-null    float64
 9   geocodes.main.longitude           129 non-null    float64
 10  geocodes.roof.latitude            121 non-null    float64
 11  geocodes.roof.longitude           121 non-null    float64
 12  location

In [66]:
df_merged4sq

,fsq_id,categories,chains,closed_bucket,distance,link,name_x,timezone,geocodes.main.latitude,geocodes.main.longitude,...,Unnamed: 0,name_y,longitude,latitude,free_bikes,empty_slots,station_1,station_2,station_3,station_4
0,4bb0c114f964a520085a3ce3,"[{'id': 13055, 'name': 'Fried Chicken Joint', ...",[{'id': '16d16020-992b-0132-fd75-7a163eb2a6fc'...,VeryLikelyOpen,421,/v3/places/4bb0c114f964a520085a3ce3,Chick-fil-A,America/Chicago,36.370851,-94.220982,...,0,Walmart International,-94.21960,36.36721,7,6,False,False,False,False
1,53cea5ae498e6131500c1fd9,"[{'id': 13003, 'name': 'Bar', 'short_name': 'B...",[],Unsure,803,/v3/places/53cea5ae498e6131500c1fd9,The Pedaler S Pub,America/Chicago,36.367122,-94.210646,...,0,Walmart International,-94.21960,36.36721,7,6,False,False,False,False
2,5862a1f1158b96178898ea98,"[{'id': 13148, 'name': 'French Restaurant', 's...",[],Unsure,949,/v3/places/5862a1f1158b96178898ea98,Crepes Paulette,America/Chicago,36.364644,-94.209418,...,0,Walmart International,-94.21960,36.36721,7,6,False,False,False,False
3,4b8dbf13f964a520bb0b33e3,"[{'id': 13031, 'name': 'Burger Joint', 'short_...",[{'id': 'ab4ae740-d68a-012e-5619-003048cad9da'...,VeryLikelyOpen,452,/v3/places/4b8dbf13f964a520bb0b33e3,Wendy's,America/Chicago,36.363141,-94.219442,...,0,Walmart International,-94.21960,36.36721,7,6,False,False,False,False
4,5441b307498e6bd51dec7685,"[{'id': 13003, 'name': 'Bar', 'short_name': 'B...",[],Unsure,803,/v3/places/5441b307498e6bd51dec7685,Bike Rack Brewing Co.,America/Chicago,36.367230,-94.210696,...,0,Walmart International,-94.21960,36.36721,7,6,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,4f442ae019836ed00192a882,"[{'id': 13025, 'name': 'Wine Bar', 'short_name...",[],LikelyOpen,295,/v3/places/4f442ae019836ed00192a882,Coors Brewing Co.,America/Chicago,36.345581,-94.192534,...,4,Walmart Training / Redbird Office Building,-94.19562,36.34648,6,6,False,False,False,True
125,4f3274f519836c91c7d9a735,"[{'id': 13025, 'name': 'Wine Bar', 'short_name...",[],LikelyOpen,412,/v3/places/4f3274f519836c91c7d9a735,Constellation Wines,America/Chicago,36.343320,-94.193207,...,4,Walmart Training / Redbird Office Building,-94.19562,36.34648,6,6,False,False,False,True
126,4c371c7d3849c928b611bdb1,"[{'id': 13034, 'name': 'Café', 'short_name': '...",[],Unsure,809,/v3/places/4c371c7d3849c928b611bdb1,The Gathering Place,America/Chicago,36.346223,-94.186574,...,4,Walmart Training / Redbird Office Building,-94.19562,36.34648,6,6,False,False,False,True
127,4d8746895e70224b58ef4509,"[{'id': 13035, 'name': 'Coffee Shop', 'short_n...",[],Unsure,810,/v3/places/4d8746895e70224b58ef4509,Idea Perk,America/Chicago,36.346223,-94.186574,...,4,Walmart Training / Redbird Office Building,-94.19562,36.34648,6,6,False,False,False,True


In [69]:
df_merged4sq.rename(columns={"Unnamed: 0": "station"}, inplace=True)

df_merged4sq

,fsq_id,categories,chains,closed_bucket,distance,link,name_x,timezone,geocodes.main.latitude,geocodes.main.longitude,...,station,name_y,longitude,latitude,free_bikes,empty_slots,station_1,station_2,station_3,station_4
0,4bb0c114f964a520085a3ce3,"[{'id': 13055, 'name': 'Fried Chicken Joint', ...",[{'id': '16d16020-992b-0132-fd75-7a163eb2a6fc'...,VeryLikelyOpen,421,/v3/places/4bb0c114f964a520085a3ce3,Chick-fil-A,America/Chicago,36.370851,-94.220982,...,0,Walmart International,-94.21960,36.36721,7,6,False,False,False,False
1,53cea5ae498e6131500c1fd9,"[{'id': 13003, 'name': 'Bar', 'short_name': 'B...",[],Unsure,803,/v3/places/53cea5ae498e6131500c1fd9,The Pedaler S Pub,America/Chicago,36.367122,-94.210646,...,0,Walmart International,-94.21960,36.36721,7,6,False,False,False,False
2,5862a1f1158b96178898ea98,"[{'id': 13148, 'name': 'French Restaurant', 's...",[],Unsure,949,/v3/places/5862a1f1158b96178898ea98,Crepes Paulette,America/Chicago,36.364644,-94.209418,...,0,Walmart International,-94.21960,36.36721,7,6,False,False,False,False
3,4b8dbf13f964a520bb0b33e3,"[{'id': 13031, 'name': 'Burger Joint', 'short_...",[{'id': 'ab4ae740-d68a-012e-5619-003048cad9da'...,VeryLikelyOpen,452,/v3/places/4b8dbf13f964a520bb0b33e3,Wendy's,America/Chicago,36.363141,-94.219442,...,0,Walmart International,-94.21960,36.36721,7,6,False,False,False,False
4,5441b307498e6bd51dec7685,"[{'id': 13003, 'name': 'Bar', 'short_name': 'B...",[],Unsure,803,/v3/places/5441b307498e6bd51dec7685,Bike Rack Brewing Co.,America/Chicago,36.367230,-94.210696,...,0,Walmart International,-94.21960,36.36721,7,6,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,4f442ae019836ed00192a882,"[{'id': 13025, 'name': 'Wine Bar', 'short_name...",[],LikelyOpen,295,/v3/places/4f442ae019836ed00192a882,Coors Brewing Co.,America/Chicago,36.345581,-94.192534,...,4,Walmart Training / Redbird Office Building,-94.19562,36.34648,6,6,False,False,False,True
125,4f3274f519836c91c7d9a735,"[{'id': 13025, 'name': 'Wine Bar', 'short_name...",[],LikelyOpen,412,/v3/places/4f3274f519836c91c7d9a735,Constellation Wines,America/Chicago,36.343320,-94.193207,...,4,Walmart Training / Redbird Office Building,-94.19562,36.34648,6,6,False,False,False,True
126,4c371c7d3849c928b611bdb1,"[{'id': 13034, 'name': 'Café', 'short_name': '...",[],Unsure,809,/v3/places/4c371c7d3849c928b611bdb1,The Gathering Place,America/Chicago,36.346223,-94.186574,...,4,Walmart Training / Redbird Office Building,-94.19562,36.34648,6,6,False,False,False,True
127,4d8746895e70224b58ef4509,"[{'id': 13035, 'name': 'Coffee Shop', 'short_n...",[],Unsure,810,/v3/places/4d8746895e70224b58ef4509,Idea Perk,America/Chicago,36.346223,-94.186574,...,4,Walmart Training / Redbird Office Building,-94.19562,36.34648,6,6,False,False,False,True


Provide model output and an interpretation of the results. 

In [71]:
import statsmodels.api as sm

X = df_merged4sq[["station"]]
Y = df_merged4sq["empty_slots"]

# Add constant for intercept
X = sm.add_constant(X)

# Fit the regression model
model = sm.OLS(Y, X).fit()

# Print the summary
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:            empty_slots   R-squared:                       0.401
Model:                            OLS   Adj. R-squared:                  0.397
Method:                 Least Squares   F-statistic:                     85.14
Date:                Wed, 16 Jul 2025   Prob (F-statistic):           7.82e-16
Time:                        18:46:06   Log-Likelihood:                -190.22
No. Observations:                 129   AIC:                             384.4
Df Residuals:                     127   BIC:                             390.2
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          5.6182      0.144     38.893      0.0

In [72]:
import statsmodels.api as sm

X = df_merged4sq[["distance"]]
Y = df_merged4sq["empty_slots"]

# Add constant for intercept
X = sm.add_constant(X)

# Fit the regression model
model = sm.OLS(Y, X).fit()

# Print the summary
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:            empty_slots   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    0.9470
Date:                Wed, 16 Jul 2025   Prob (F-statistic):              0.332
Time:                        18:48:20   Log-Likelihood:                -222.83
No. Observations:                 129   AIC:                             449.7
Df Residuals:                     127   BIC:                             455.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.2544      0.380     11.206      0.0

In [73]:
df_mergedyelp

,id,alias,name_x,image_url,is_closed,url,review_count,categories,rating,transactions,...,attributes.menu_url,attributes.open24_hours,attributes.waitlist_reservation,station,Unnamed: 0,name_y,longitude,latitude,free_bikes,empty_slots
0,yfzJiR-UbNVT6_tOd4neYA,meld-kitchen-sandwich-bar-bentonville-2,MELD Kitchen + Sandwich Bar,https://s3-media0.fl.yelpcdn.com/bphoto/fQ6cWi...,False,https://www.yelp.com/biz/meld-kitchen-sandwich...,276,"[{'alias': 'tradamerican', 'title': 'American'...",4.8,"['pickup', 'delivery']",...,https://www.meldkitchen.com/menu,NaN,NaN,0,0,Walmart International,-94.21960,36.36721,7,6
1,abN69kKRGfG4-SyMJhWd7w,tavola-trattoria-bentonville,Tavola Trattoria,https://s3-media0.fl.yelpcdn.com/bphoto/pezUvd...,False,https://www.yelp.com/biz/tavola-trattoria-bent...,633,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.1,['delivery'],...,NaN,NaN,NaN,0,0,Walmart International,-94.21960,36.36721,7,6
2,E_-cU7G4sVlE3NkUNQoPmA,table-on-6th-bentonville,Table on 6th,https://s3-media0.fl.yelpcdn.com/bphoto/SCStmE...,False,https://www.yelp.com/biz/table-on-6th-bentonvi...,200,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",4.3,[],...,https://tableon6th.com/menus/,NaN,NaN,0,0,Walmart International,-94.21960,36.36721,7,6
3,hgOJKwMkQncCiVan9e5vSQ,pedalers-pub-bentonville,Pedaler's Pub,https://s3-media0.fl.yelpcdn.com/bphoto/_AY8UC...,False,https://www.yelp.com/biz/pedalers-pub-bentonvi...,429,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,['delivery'],...,http://pedalerspub.com/menu.html,NaN,NaN,0,0,Walmart International,-94.21960,36.36721,7,6
4,IoBL7iqyay0KYeVZtNJL1A,bentonville-taco-and-tamale-bentonville,Bentonville Taco & Tamale,https://s3-media0.fl.yelpcdn.com/bphoto/olyT5F...,False,https://www.yelp.com/biz/bentonville-taco-and-...,267,"[{'alias': 'tex-mex', 'title': 'Tex-Mex'}, {'a...",4.0,['restaurant_reservation'],...,NaN,NaN,True,0,0,Walmart International,-94.21960,36.36721,7,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237,FsANyp1YR3oTIabpbAEy3g,freddys-frozen-custard-and-steakburgers-rogers,Freddy's Frozen Custard & Steakburgers,https://s3-media0.fl.yelpcdn.com/bphoto/7Sij75...,False,https://www.yelp.com/biz/freddys-frozen-custar...,66,"[{'alias': 'hotdog', 'title': 'Hot Dogs'}, {'a...",3.4,[],...,http://www.freddysusa.com/menu,NaN,NaN,4,4,Walmart Training / Redbird Office Building,-94.19562,36.34648,6,6
238,HDEF5AxD1H0awOHzpvhMdg,mcalisters-deli-bentonville,McAlister's Deli,https://s3-media0.fl.yelpcdn.com/bphoto/AYd76C...,False,https://www.yelp.com/biz/mcalisters-deli-bento...,54,"[{'alias': 'delis', 'title': 'Delis'}, {'alias...",3.3,"['pickup', 'delivery']",...,https://www.mcalistersdeli.com/menu,NaN,NaN,4,4,Walmart Training / Redbird Office Building,-94.19562,36.34648,6,6
239,TuLjnDH7K5tnc69eWjQMiw,king-burrito-bentonville-4,King Burrito,https://s3-media0.fl.yelpcdn.com/bphoto/6mr9g9...,False,https://www.yelp.com/biz/king-burrito-bentonvi...,88,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",3.7,"['pickup', 'delivery']",...,NaN,NaN,NaN,4,4,Walmart Training / Redbird Office Building,-94.19562,36.34648,6,6
240,DAkeRMiSBI9ucZODBoiFfg,great-harvest-nwa-bentonville,Great Harvest NWA,https://s3-media0.fl.yelpcdn.com/bphoto/AVCsPQ...,False,https://www.yelp.com/biz/great-harvest-nwa-ben...,0,"[{'alias': 'cafes', 'title': 'Cafes'}, {'alias...",0.0,[],...,NaN,NaN,NaN,4,4,Walmart Training / Redbird Office Building,-94.19562,36.34648,6,6


In [60]:
import statsmodels.api as sm

X = df_mergedyelp[["station"]]
Y = df_mergedyelp["empty_slots"]

# Add constant for intercept
X = sm.add_constant(X)

# Fit the regression model
model = sm.OLS(Y, X).fit()

# Print the summary
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:            empty_slots   R-squared:                       0.013
Model:                            OLS   Adj. R-squared:                  0.009
Method:                 Least Squares   F-statistic:                     3.103
Date:                Wed, 16 Jul 2025   Prob (F-statistic):             0.0794
Time:                        16:36:57   Log-Likelihood:                -400.11
No. Observations:                 242   AIC:                             804.2
Df Residuals:                     240   BIC:                             811.2
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          5.1669      0.140     36.954      0.0

As expected, there is not a strong corellation when using this data because there is a lot of information that is unavailable (or simply too difficult to get with time and ability contraints). There is no real way to know where people are going on the bikes, and to get more useful information one would need data from different times of the day or different times of the year. My guess is the bikes are used and move largely from station to station during work hours because of the Walmart presence in Bentonville. 

# Stretch